In [1]:
import numpy as np 
import pandas as pd 
import math

In [3]:
print(f"Ghost_real : {len(df_G_Real)}, Unique MATCLE : {len(df_G_Real['MATCLE'].unique())},  Unique 10.000 : {len(df_G_Real.iloc[:5000,17].unique())}")
print(f"Time_real : {len(df_T_Real)}, Unique MATCLE : {len(df_T_Real['MATCLE'].unique())},  Unique 10.000 : {len(df_T_Real.iloc[:5000,10].unique())}")
print(f"Misc_real : {len(df_M_Real)}, Unique MATCLE : {len(df_M_Real['MATCLE'].unique())},  Unique 10.000 : {len(df_M_Real.iloc[:5000,21].unique())}")

Ghost_real : 926366, Unique MATCLE : 6590,  Unique 10.000 : 207
Time_real : 701441, Unique MATCLE : 14422,  Unique 10.000 : 436
Misc_real : 60228, Unique MATCLE : 42124,  Unique 10.000 : 2631


(48-57) : numbers

(65-90) : uppercase

(97-122) : lowercase 

In [3]:
# def anonym(MAT) :
#     ascii_codes = [ord(char) for char in MAT]
#     encoding_n = ascii_codes[:]
#     print(ascii_codes[0])
#     if (ascii_codes[0] in range(65,90)) :
#         encoding_n[0] = int((100*math.log(ascii_codes[0])/6 ) +5)
#         print('lett')
#     else : 
#         encoding_n[0] = int((100*math.log(ascii_codes[0])/7 ) +1)
#         print('nbr')

#     for i in range(1,6):
#         encoding_n[i]=(int((100*math.log(ascii_codes[i])/7)-1)-i)
#         print(f'done{i}')
       

#     for j,e in enumerate(encoding_n) : 
#         print(j,e,'\\')
      
#     print('\n ----')
    
    
#     #result = [chr(code) for code in encoding_n ]
#     f_result = ''.join(chr(code) for code in encoding_n)
#     return f_result

# Full Pipeline for Fraud & Real Data

We will define a set of functions that perform data preparation in a Final Usable state for the classifier models.
We define these steps for each Fraud type Data : 
- Anonymization step of Batch Fraud data
- Formatting of Batch Fraud Data label (Fraud column values)
- Appending Batch Fraud Data to last update Clean Fraud Data
- Creating a 1/5 balanced dataset of Fraud data + Real data, based on DataFrames input


INPUT : 
- DataFrame of Fraud data type i (raw generated in csv format)
- DataFrame of Fraud data type i (anonymized full usable data)

## Anonymization Functions

In [2]:
## GHOST & TIMESHEET DATA

def encodePersonID_Gh_Time(df,L) :
    if len(df['PersonID'].unique())>len(L) :
        return 'Not enough IDs to assign to dataset'
    i=0
    j=0
    k=-1
    while i <len(df) : 
        val = df.iloc[i,0]
        k+=1
        print(k)
        while j<len(df) and val == df.iloc[j,0] : 
            df.iloc[j,0] = L.iloc[k,0]
            j+=1
        i=j
        # L.iloc[1:].reset_index(drop=True)
        # L.tail(-1).reset_index(drop=True, inplace = True)
        L.drop(index=L.index[0]).reset_index(drop=True)
        # print(f" i,j = {i,j} , val[i] = {val},  , L value =  {L.iloc[i,0]} , df id value =  {df.iloc[i,0]}" )
    return df,L.iloc[k:,]
    # return df,L 

In [16]:
## Misclass

def encodePersonID_Misc(df,L) : 
    for i in range(len(df)) :
        # print(f" i = {i} , L size =  {L.iloc[i,0]} , df size =  {df.iloc[i,0]}" )
        df.iloc[i,0] = L.iloc[i,0]
        # L.iloc[1:].reset_index(drop=True)
        # L.tail(-1).reset_index(drop=True, inplace = True)
        L.drop(index=L.index[0]).reset_index(drop=True)
        print(L.shape)
    return df,L.iloc[len(df):,]
    # return df,L

## Main Process Functions

In [4]:
# GHOST & TIMESHEET

def append_Last_Update_Fraud_GH_Time(df_fraud_new,df_Fraud_last_update,df_real_data,L,Fraud_Type) : 
    df_anonym, L_upd = encodePersonID_Gh_Time(df_fraud_new, L)

    # Modifications on Fraud data
    df_anonym.columns = df_anonym.columns.str.strip()
    if 'Explanation' in df_anonym.columns : 
        df_anonym.drop('Explanation', inplace=True, axis=1)
    if 'Explanation' in df_Fraud_last_update.columns : 
        df_Fraud_last_update.drop('Explanation', inplace=True, axis=1)

    
    # Labelization
    if Fraud_Type == "Ghost" : 
        df_anonym["Fraud"] = 1
        df_Fraud_last_update["Fraud"] = 1
    elif Fraud_Type == "Timesheet" : 
        df_anonym["Fraud"] = 2
        df_Fraud_last_update["Fraud"] = 2

    # Labelization real data
    if 'Fraud' not in df_real_data.columns : 
        df_real_data["Fraud"] = 0
    
    # Appending to latest update fraud data
    df_Fraud_upd = pd.concat([df_anonym,df_Fraud_last_update],ignore_index= True)
    
    # Appending to Real data
    n = df_Fraud_upd.shape[0]*5
    print(n)
    df_full = pd.concat([df_Fraud_upd,df_real_data.iloc[:n,:]],ignore_index= True)

    return L_upd, df_Fraud_upd, df_full
    

In [17]:
# Misclassif

def append_Last_Update_Fraud_Misc(df_fraud_new,df_Fraud_last_update,df_real_data,L) : 
    df_anonym, L_upd = encodePersonID_Misc(df_fraud_new, L)
    print(L.shape)
    
    # Treatments for json file
    df_anonym.columns = df_anonym.columns.str.strip()
    if 'Explanation' in df_anonym.columns : 
        df_anonym.drop('Explanation', inplace=True, axis=1)
    # df_anonym.drop('Position', inplace=True, axis=1)
    
    # Labelization
    df_anonym["Fraud"] = 3

    # Appending to latest update fraud data
    df_Fraud_upd_M = pd.concat([df_anonym,df_Fraud_last_update],ignore_index= True)
    
    # Appending to Real data
    n = df_Fraud_upd_M.shape[0]*5
    df_full_M = pd.concat([df_Fraud_upd_M,df_real_data.iloc[:n,:]],ignore_index= True)

    # Saving
    L_upd.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\MATCLE_Encoding\L_upd_M2.csv",index=False)
    df_Fraud_upd_M.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\0. Classifier Data\df_fraud_last_upd\upd_M_2.csv",index=False)
    df_full_M.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\0. Classifier Data\df_full\df_full_M2.csv",index=False)
    
    return L_upd, df_Fraud_upd_M, df_full_M
    

# Pipeline Execution

## Ghost Fraud

### Load

In [108]:
file_name_g = "G2-5-6"
# file_name_g = "json_ghost"

In [109]:
## Load New Fraud Data
path_fr_g=r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\1. ADAPT-LLM\Data_Fraud\Fraud_Clean_data_pre-anon\Ghost\\"
df_fraud_new_g = pd.read_json(path_fr_g + file_name_g + ".json")

In [110]:
## Load real data
df_real_G = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\Anonym Real Data\df_G_Real_anonym.csv")
## Load Last updates
df_fraud_last_upd_g = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\0. Classifier Data\df_fraud_last_upd\Archive\upd_G_1.csv")
L_G = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\MATCLE_Encoding\ArchiveBis\L_upd_G_1.csv")

### Process

In [115]:
L_upd_G

,Coded
39,700074
40,700075
41,700076
42,700077
43,700078
...,...
960,700995
961,700996
962,700997
963,700998


In [111]:
L_upd_G, df_Fraud_upd_G, df_full_G = append_Last_Update_Fraud_GH_Time(df_fraud_new_g, df_fraud_last_upd_g, df_real_G, L_G,"Ghost")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
3605


In [112]:
df_full_G

,PersonID,Document_Type,Document_name,Documents_Deposit,Deposit_Platform,Absence_CODE,Absence_Reason,Seniority_in_Group (in days),Leave_Date,Physical_Leave,...,MOTIFA,SEN_GRP_DUR_int,DATSOR,PHYSOR,DATFIN,DATENT,QUALIT,IDCY00,IDXCOM,MATCLE
0,700035.0,DOC11,Bank account number,Scanned & Validated,F,ABS,Unjustified Absence,100.0,31/12/2050,31/12/2050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,700035.0,DOC01,Work Contract,No deposit,F,ABS,Unjustified Absence,100.0,31/12/2050,31/12/2050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,700035.0,DOC02,Birth Certificate,No deposit,F,ABS,Unjustified Absence,100.0,31/12/2050,31/12/2050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,700035.0,DOC01,Work Contract,Scanned & Validated,F,ABS,Unjustified Absence,500.0,31/12/2050,31/12/2050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,700035.0,DOC13,Marriage Certificate copy,Scanned & Validated,F,ABS,Unjustified Absence,500.0,31/12/2050,31/12/2050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MAN,10288.0,2050-12-31 01:00:00,2050-12-31 23:00:00,2023-01-09 01:00:00,2022-10-31 01:00:00,Mrs,TWW,Unspecified,40871.0
4322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MAN,1771.0,2021-03-01 01:00:00,2050-12-31 23:00:00,2023-01-09 01:00:00,2016-04-25 01:00:00,Mrs,TWW,Unspecified,40872.0
4323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MAN,10288.0,2050-12-31 01:00:00,2050-12-31 23:00:00,2023-01-09 01:00:00,2022-10-31 01:00:00,Mrs,TWW,Unspecified,40872.0
4324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MAN,1771.0,2021-03-01 01:00:00,2050-12-31 23:00:00,2023-01-09 01:00:00,2016-04-25 01:00:00,Mrs,TWW,Unspecified,40873.0


In [113]:
df_Fraud_upd_G

,PersonID,Document_Type,Document_name,Documents_Deposit,Deposit_Platform,Absence_CODE,Absence_Reason,Seniority_in_Group (in days),Leave_Date,Physical_Leave,...,CODSOR,Current_company,Leave_Reason,Transfer_Reason,Transfer_company,Leave_DATE,Fraud,ID,Seniority_in_Group,Transfer_Reason
0,700035,DOC11,Bank account number,Scanned & Validated,F,ABS,Unjustified Absence,100.0,31/12/2050,31/12/2050,...,Currently IN,EXM,FPE,Not Transferred,UNSPECFIED,NaN,1,NaN,NaN,NaN
1,700035,DOC01,Work Contract,No deposit,F,ABS,Unjustified Absence,100.0,31/12/2050,31/12/2050,...,Currently IN,EXM,FPE,Not Transferred,UNSPECFIED,NaN,1,NaN,NaN,NaN
2,700035,DOC02,Birth Certificate,No deposit,F,ABS,Unjustified Absence,100.0,31/12/2050,31/12/2050,...,Currently IN,EXM,FPE,Not Transferred,UNSPECFIED,NaN,1,NaN,NaN,NaN
3,700035,DOC01,Work Contract,Scanned & Validated,F,ABS,Unjustified Absence,500.0,31/12/2050,31/12/2050,...,Currently IN,EXM,DEM,Not Transferred,UNSPECFIED,NaN,1,NaN,NaN,NaN
4,700035,DOC13,Marriage Certificate copy,Scanned & Validated,F,ABS,Unjustified Absence,500.0,31/12/2050,31/12/2050,...,Currently IN,EXM,DEM,Not Transferred,UNSPECFIED,NaN,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,700028,DOC11,Bank account number,Scanned & Validated,O,1000,Absence Entry/Leave,180.0,31/12/2050,31/12/2050,...,Currently IN,EXM,PROB,NaN,UNSPECFIED,NaN,1,NaN,NaN,Not Transferred
717,700028,DOC08,Certified Copy of National ID Card,Scanned & Validated,O,NaN,Unjustified Absence,200.0,31/12/2050,31/12/2050,...,Currently IN,EXM,Currently IN,NaN,UNSPECFIED,NaN,1,NaN,NaN,Not Transferred
718,700028,DOC01,Work Contract,Scanned & Validated,O,ABS,Unjustified Absence,200.0,31/12/2050,31/12/2050,...,Currently IN,EXM,DEM,NaN,UNSPECFIED,NaN,1,NaN,NaN,Not Transferred
719,700028,DOC07,Certified Diploma copy,Scanned & Validated,O,ABS,Unjustified Absence,200.0,31/12/2050,31/12/2050,...,Currently IN,EXM,DEM,NaN,UNSPECFIED,NaN,1,NaN,NaN,Not Transferred


### Save updates

In [116]:
L_upd_G.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\MATCLE_Encoding\L_upd_G_2.csv",index=False)
df_Fraud_upd_G.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\0. Classifier Data\df_fraud_last_upd\upd_G_2.csv",index=False)
df_full_G.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\0. Classifier Data\df_full\df_full_G_2.csv",index=False)    

## Timesheet Fraud

### Load

In [88]:
file_name_t = "T1-2-19-24-3-4-15-8-9-17-18"
# file_name_t = "T6&10_plus_models_testing_kpis_output"

In [89]:
## Load New Fraud Data
path_fr_t= r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\1. ADAPT-LLM\Data_Fraud\Fraud_Clean_data_pre-anon\Timesheet\\"
# df_fraud_new_t = pd.read_csv(path_fr_t+file_name_t+".csv")
df_fraud_new_t = pd.read_json(path_fr_t+file_name_t+".json")

In [90]:
df_fraud_new_t

,PersonID,SOURCE,Payroll_Period,Absence_Reason,Absence_CODE,Absence_Start_Date,Absence_End_Date,Absence_Duration,Acquired_Leave_days,Taken_Leave_days,Remaining_Leave_days,Following_year_reported_Leave,Explanation
0,500001,R,03/2023,Payed Leave,CPN,10/03/2023,14/03/2023,5.0,NaN,47,2,1.0,The employee is marked as having taken 5 paid ...
1,500005,R,02/2023,Sick Leave,MAL,12/02/2023,15/02/2023,3.0,20.0,3,17,0.0,"This employee has 20 acquired leave days, his ..."
2,500004,R,02/2023,Sick Leave,MAL,02/02/2023,06/02/2023,4.0,20.0,4,16,0.0,"This employee has 20 acquired leave days, his ..."
3,123456,R,08/2023,Payed Leave,CPN,05/08/2023,12/08/2023,8.0,18.0,8,10,0.0,None
4,654321,R,08/2023,Payed Leave,CPN,20/08/2023,27/08/2023,8.0,10.0,8,12,0.0,"This employee has acquired 18 leave days, take..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
617,500027,STD,04/2023,Sick Leave,MAL,03/04/2023,12/04/2023,9.0,17.8,9,8.8,0.0,NaN
618,500004,R,08/2023,Sick Leave,MAL,01/08/2023,05/08/2023,5.0,20.0,5,15,0.0,NaN
619,500004,R,08/2023,Sick Leave,MAL,12/08/2023,17/08/2023,6.0,20.0,6,9,0.0,NaN
620,500016,STD,07/2023,Payed Leave,CPN,10/07/2023,15/07/2023,6.0,5.0,6,0,2.0,NaN


In [91]:
df_fraud_last_upd_t

,PersonID,SOURCE,Payroll_Period,Absence_Reason,Absence_CODE,Absence_Start_Date,Absence_End_Date,Absence_Duration,Acquired_Leave_days,Taken_Leave_days,Remaining_Leave_days,Following_year_reported_Leave,Fraud
0,500000,STD,10/2023,Payed Leave,CPN,02/10/2023,04/10/2023,2,22,27,2,0.0,2
1,500001,R,02/2022,Sick Leave,MAL,02/02/2022,06/10/2023,4,24,2,22,0.0,2
2,500002,STD,04/2023,Sick Leave,MAL,03/04/2023,12/04/2023,9,17.8,4,13.8,0.0,2
3,500003,HR,05/2023,Payed Leave,CPN,05/10/2023,05/15/2023,6,18,10,8,0.0,2
4,500004,FIN,06/2023,Sick Leave,MAL,06/01/2023,06/05/2023,5,20,3,17,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,500068,FIN,08/2023,Sick Leave,MAL,05/08/2023,10/08/2023,6,17.0,6,9.0,0.0,2
121,500068,FIN,08/2023,Sick Leave,MAL,15/08/2023,20/08/2023,6,9.0,6,6.0,0.0,2
122,500069,HR,09/2023,Payed Leave,CPN,01/09/2023,01/09/2023,1,16.0,1,15.0,0.0,2
123,500069,HR,09/2023,Payed Leave,CPN,05/09/2023,10/09/2023,6,15.0,6,8.0,0.0,2


In [92]:
## Load real data
df_real_T = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\Anonym Real Data\df_T_Real_anonym.csv")
## Load last updates
df_fraud_last_upd_t = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\0. Classifier Data\df_fraud_last_upd\Archive\upd_T_1.csv")
L_T = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\MATCLE_Encoding\ArchiveBis\L_upd_T1.csv")

In [94]:
# L_T.iloc[1:].reset_index(drop=True)
L_upd_T

,Coded
480,500610
481,500611
482,500612
483,500613
484,500614
...,...
865,500995
866,500996
867,500997
868,500998


### Process

In [93]:
L_upd_T, df_Fraud_upd_T, df_full_T = append_Last_Update_Fraud_GH_Time(df_fraud_new_t, df_fraud_last_upd_t, df_real_T, L_T, "Timesheet")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [95]:
df_Fraud_upd_T["PersonID"].unique()

array([500130, 500131, 500132, 500133, 500134, 500135, 500136, 500137,
       500138, 500139, 500140, 500141, 500142, 500143, 500144, 500145,
       500146, 500147, 500148, 500149, 500150, 500151, 500152, 500153,
       500154, 500155, 500156, 500157, 500158, 500159, 500160, 500161,
       500162, 500163, 500164, 500165, 500166, 500167, 500168, 500169,
       500170, 500171, 500172, 500173, 500174, 500175, 500176, 500177,
       500178, 500179, 500180, 500181, 500182, 500183, 500184, 500185,
       500186, 500187, 500188, 500189, 500190, 500191, 500192, 500193,
       500194, 500195, 500196, 500197, 500198, 500199, 500200, 500201,
       500202, 500203, 500204, 500205, 500206, 500207, 500208, 500209,
       500210, 500211, 500212, 500213, 500214, 500215, 500216, 500217,
       500218, 500219, 500220, 500221, 500222, 500223, 500224, 500225,
       500226, 500227, 500228, 500229, 500230, 500231, 500232, 500233,
       500234, 500235, 500236, 500237, 500238, 500239, 500240, 500241,
      

In [68]:
df_full_T.columns

Index(['PersonID', 'SOURCE', 'Payroll_Period', 'Absence_Reason',
       'Absence_CODE', 'Absence_Start_Date', 'Absence_End_Date',
       'Absence_Duration', 'Acquired_Leave_days', 'Taken_Leave_days',
       'Remaining_Leave_days', 'Following_year_reported_Leave', 'Fraud',
       'MATCLE'],
      dtype='object')

In [96]:
df_full_T

,PersonID,SOURCE,Payroll_Period,Absence_Reason,Absence_CODE,Absence_Start_Date,Absence_End_Date,Absence_Duration,Acquired_Leave_days,Taken_Leave_days,Remaining_Leave_days,Following_year_reported_Leave,Fraud,MATCLE
0,500130.0,R,03/2023,Payed Leave,CPN,10/03/2023,14/03/2023,5.0,NaN,47,2,1.0,2,NaN
1,500131.0,R,02/2023,Sick Leave,MAL,12/02/2023,15/02/2023,3.0,20.0,3,17,0.0,2,NaN
2,500132.0,R,02/2023,Sick Leave,MAL,02/02/2023,06/02/2023,4.0,20.0,4,16,0.0,2,NaN
3,500133.0,R,08/2023,Payed Leave,CPN,05/08/2023,12/08/2023,8.0,18.0,8,10,0.0,2,NaN
4,500134.0,R,08/2023,Payed Leave,CPN,20/08/2023,27/08/2023,8.0,10.0,8,12,0.0,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5599,NaN,R,"""MT202301""",Payed Leave,CPN,2022-12-28 01:00:00,2022-12-28 01:00:00,0.0,18.0,8.5,0.0,9.5,0,20293.0
5600,NaN,R,"""MT202301""",Payed Leave,CPN,2022-12-28 01:00:00,2022-12-28 01:00:00,0.0,18.0,14.0,0.0,13.5,0,20293.0
5601,NaN,R,"""MT202301""",Payed Leave,CPN,2022-12-28 01:00:00,2022-12-28 01:00:00,0.0,18.0,6.0,0.0,25.5,0,20293.0
5602,NaN,R,"""MT202301""",Payed Leave,CPN,2022-12-28 01:00:00,2022-12-28 01:00:00,0.0,18.0,22.0,0.0,21.5,0,20293.0


### Save updates

In [98]:
L_upd_T.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\MATCLE_Encoding\ArchiveBis\L_upd_T2.csv", index=False)
df_Fraud_upd_T.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\0. Classifier Data\df_fraud_last_upd\upd_T_2.csv",index=False)
df_full_T.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\0. Classifier Data\df_full\Archive\df_full_T2.csv",index=False)    

## Misclassification Fraud

### Load

In [18]:
file_name_m = "M9--21"  # with no .json
# file_name_m = "M1-8"  # with no .json

In [19]:
## Load New Fraud Data
path_fr_m = r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\1. ADAPT-LLM\Data_Fraud\Fraud_Clean_data_pre-anon\Misclassification\\"
df_fraud_new_m = pd.read_json(path_fr_m+file_name_m+".json")

In [20]:
## Load real data
df_real_M = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\Anonym Real Data\df_M_Real_anonym_uniformized.csv")

## Load last updates
df_fraud_last_upd_m = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\0. Classifier Data\df_fraud_last_upd\Archive\upd_M_1.csv")
L_M = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Fraud Detection\4. Models\2. Classifier\MATCLE_Encoding\ArchiveBis\L_upd_M1.csv")

In [21]:
df_fraud_last_upd_m

,PersonID,Seniority_previous_company(N-2) (in days),Contract_in_group_duration (in days),Seniority_previous_company(N-1) (in days),Seniority_in_Group (in days),Contract_Date,Leave_Date,Entry_Date_Company_1,Sex,Contract Type,...,Position,Classification,Work_form,CODSOR,Original_company,Leave_Category,Leave_Reason,Transfer_Reason,Transfer_company,Fraud
0,400000,0,16485,0,12400,18/01/2017,2050-12-31,01/01/1753,Mrs,DD,...,Employee,Employee E4,Full Time,Currently IN,TMG,Unspecified,Currently IN,Not Transferred,UNSPECFIED,3
1,400001,0,12667,NaN,11777,03/10/2018,2050-12-31,01/01/1753,Mr,DI,...,Supervisor,Supervisor,Full Time,Currently IN,TWW,Unspecified,Currently IN,Not Transferred,UNSPECFIED,3
2,400002,0,14400,0,13000,29/05/2015,2050-12-31,01/01/1753,Miss,DI,...,Manager/Director,Executive C3,Part Time,Currently IN,EUR,Unspecified,Currently IN,Not Transferred,UNSPECFIED,3
3,400003,0,12000,0,14880,05/04/2010,2050-12-31,01/01/1753,Mr,DI,...,Supervisor,Supervisor,Full Time,Currently IN,ITI,Unspecified,Currently IN,Not Transferred,UNSPECFIED,3
4,400004,0,11990,"1096,00",12900,15/03/2022,2050-12-31,15/03/2019,Miss,DD,...,Employee,Employee E1,Full Time,Currently IN,PRO,Transfer,TRA,Voluntary,TWW,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,400044,0,11111,0,12330,29/03/2017,31/12/2050,01/01/1753,Mr,DI,...,NaN,Employee E3,Full Time,Currently IN,IIS,Unspecified,Currently IN,Not Transferred,UNSPECFIED,3
374,400046,0,10000,1338,10000,14/07/2021,31/12/2050,14/11/2017,Mrs,DI,...,NaN,Employee E2,Full Time,Currently IN,SBS,End_Contract,End of Contract,Organisation,IMI,3
375,400048,0,10990,1096,11900,15/03/2022,31/12/2050,15/03/2019,Miss,DI,...,NaN,Employee E1,Full Time,Currently IN,ATX,Transfer,AUTRE,Voluntary,IIS,3
376,400050,0,13770,366,11200,02/02/2022,31/12/2050,01/02/2021,Mr,DD,...,NaN,Executive C2,Full Time,Currently IN,AFA,Transfer,End of Contract,Organisation,EXM,3


In [22]:
df_fraud_new_m

,PersonID,Seniority_previous_company(N-2) (in days),Contract_in_group_duration (in days),Seniority_previous_company(N-1) (in days),Seniority_in_Group (in days),Contract_Date,Leave_Date,Entry_Date_Company_1,Sex,Contract Type,...,Classification,Work_form,CODSOR,Original_company,Leave_Category,Leave_Reason,Transfer_Reason,Transfer_company,Explanation,Contract_Type
0,400032,0,16485,0,12400,18/01/2017,2050-12-31,01/01/1753,Mrs,DD,...,Employee E4,Full Time,Currently IN,TMG,Unspecified,Currently IN,Not Transferred,UNSPECFIED,"Contract Entry Date is 2017, she's been in the...",NaN
1,400034,0,12667,0,11777,03/10/2018,2050-12-31,01/01/1753,Mr,DI,...,Supervisor,Full Time,Currently IN,TWW,Unspecified,Currently IN,Not Transferred,UNSPECFIED,"Contract Entry Date is 2018, he's been in the ...",NaN
2,10001,3650,17280,0,1200,01/01/2017,2050-12-31,01/01/1753,Male,DI,...,Executive C3,Full Time,Currently IN,UNSPECIFIED,Unspecified,Currently IN,Not Transferred,UNSPECFIED,The employee has been in the company for 9 yea...,NaN
3,400038,0,12000,0,14880,05/04/2010,2050-12-31,01/01/1753,Mr,DI,...,Supervisor,Full Time,Currently IN,ITI,Unspecified,Currently IN,Not Transferred,UNSPECFIED,"Contract Entry Date is 2010, he's been in the ...",NaN
4,400040,0,11990,"1096,00",12900,15/03/2022,2050-12-31,15/03/2019,Miss,DD,...,Employee E1,Full Time,Currently IN,PRO,Transfer,TRA,Voluntary,TWW,Employee transferred after end of contract wit...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,400054,0,12135,0,13135,10/10/2020,31/12/2050,01/01/1753,Mrs,DI,...,Mgr/Director,Full Time,Currently IN,EUR,Unspecified,Currently IN,Not Transferred,UNSPECFIED,"Contract Entry Date is 2020, she's been in the...",NaN
556,400056,0,14135,0,15135,10/10/2024,31/12/2050,01/01/1753,Mr,DI,...,Executive,Full Time,Currently IN,EUR,Unspecified,Currently IN,Not Transferred,UNSPECIFIED,"Contract Entry Date is 2024, he's been in the ...",NaN
557,400058,0,16135,0,17135,10/10/2018,31/12/2050,01/01/1753,Mr,DI,...,Executive,Full Time,Currently IN,EUR,Unspecified,Currently IN,Not Transferred,UNSPECFIED,"Contract Entry Date is 2028, he's been in the ...",NaN
558,400060,0,18135,0,19135,10/10/2016,31/12/2050,01/01/1753,Mr,DI,...,Executive,Full Time,Currently IN,EUR,Unspecified,Currently IN,Not Transferred,UNSPECFIED,"Contract Entry Date is 2016, he's been in the ...",NaN


In [23]:
L_M

,Coded
0,400335
1,400336
2,400337
3,400338
4,400339
...,...
660,400995
661,400996
662,400997
663,400998


In [30]:
# L_M = L_M.iloc[1:].reset_index(drop=True)
# L_M = L_M.tail(-1)

### Process

In [24]:
L_upd_M, df_Fraud_upd_M, df_full_M = append_Last_Update_Fraud_Misc(df_fraud_new_m, df_fraud_last_upd_m, df_real_M, L_M)

(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(665, 1)
(

In [29]:
df_Fraud_upd_M['PersonID'].unique()
# df_Fraud_upd_M

array([400335, 400336, 400337, 400338, 400339, 400340, 400341, 400342,
       400343, 400344, 400345, 400346, 400347, 400348, 400349, 400350,
       400351, 400352, 400353, 400354, 400355, 400356, 400357, 400358,
       400359, 400360, 400361, 400362, 400363, 400364, 400365, 400366,
       400367, 400368, 400369, 400370, 400371, 400372, 400373, 400374,
       400375, 400376, 400377, 400378, 400379, 400380, 400381, 400382,
       400383, 400384, 400385, 400386, 400387, 400388, 400389, 400390,
       400391, 400392, 400393, 400394, 400395, 400396, 400397, 400398,
       400399, 400400, 400401, 400402, 400403, 400404, 400405, 400406,
       400407, 400408, 400409, 400410, 400411, 400412, 400413, 400414,
       400415, 400416, 400417, 400418, 400419, 400420, 400421, 400422,
       400423, 400424, 400425, 400426, 400427, 400428, 400429, 400430,
       400431, 400432, 400433, 400434, 400435, 400436, 400437, 400438,
       400439, 400440, 400441, 400442, 400443, 400444, 400445, 400446,
      

In [25]:
df_full_M

,PersonID,Seniority_previous_company(N-2) (in days),Contract_in_group_duration (in days),Seniority_previous_company(N-1) (in days),Seniority_in_Group (in days),Contract_Date,Leave_Date,Entry_Date_Company_1,Sex,Contract Type,...,Classification,Work_form,CODSOR,Original_company,Leave_Category,Leave_Reason,Transfer_Reason,Transfer_company,Contract_Type,Fraud
0,400335,0,16485,0,12400,18/01/2017,2050-12-31,01/01/1753,Mrs,DD,...,Employee E4,Full Time,Currently IN,TMG,Unspecified,Currently IN,Not Transferred,UNSPECFIED,NaN,3
1,400336,0,12667,0,11777,03/10/2018,2050-12-31,01/01/1753,Mr,DI,...,Supervisor,Full Time,Currently IN,TWW,Unspecified,Currently IN,Not Transferred,UNSPECFIED,NaN,3
2,400337,3650,17280,0,1200,01/01/2017,2050-12-31,01/01/1753,Male,DI,...,Executive C3,Full Time,Currently IN,UNSPECIFIED,Unspecified,Currently IN,Not Transferred,UNSPECFIED,NaN,3
3,400338,0,12000,0,14880,05/04/2010,2050-12-31,01/01/1753,Mr,DI,...,Supervisor,Full Time,Currently IN,ITI,Unspecified,Currently IN,Not Transferred,UNSPECFIED,NaN,3
4,400339,0,11990,"1096,00",12900,15/03/2022,2050-12-31,15/03/2019,Miss,DD,...,Employee E1,Full Time,Currently IN,PRO,Transfer,TRA,Voluntary,TWW,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5623,14685,0,48,96470,51,2017-02-16,2012-11-21 00:00:00,2012-10-01,Mr,DI,...,Employee,Full Time,Left,TWW,VOLUNT,ABN,Not Transferred,Undefined,NaN,0
5624,14686,0,51,94871,48,2012-10-01,2017-04-05 00:00:00,2017-02-16,Mr,DI,...,Employee,Full Time,Left,TWW,VOLUNT,ABN,Not Transferred,Undefined,NaN,0
5625,14687,0,48,96470,48,2017-02-16,2017-04-05 00:00:00,2017-02-16,Mr,DI,...,Employee,Full Time,Left,TWW,VOLUNT,ABN,Not Transferred,Undefined,NaN,0
5626,14688,0,8,94864,8,2012-09-24,2012-10-02 00:00:00,2012-09-24,Mr,DI,...,Employee,Full Time,Left,TWW,VOLUNT,ABN,Not Transferred,Undefined,NaN,0


In [28]:
df_Fraud_upd_M

,PersonID,Seniority_previous_company(N-2) (in days),Contract_in_group_duration (in days),Seniority_previous_company(N-1) (in days),Seniority_in_Group (in days),Contract_Date,Leave_Date,Entry_Date_Company_1,Sex,Contract Type,...,Classification,Work_form,CODSOR,Original_company,Leave_Category,Leave_Reason,Transfer_Reason,Transfer_company,Contract_Type,Fraud
0,400335,0,16485,0,12400,18/01/2017,2050-12-31,01/01/1753,Mrs,DD,...,Employee E4,Full Time,Currently IN,TMG,Unspecified,Currently IN,Not Transferred,UNSPECFIED,NaN,3
1,400336,0,12667,0,11777,03/10/2018,2050-12-31,01/01/1753,Mr,DI,...,Supervisor,Full Time,Currently IN,TWW,Unspecified,Currently IN,Not Transferred,UNSPECFIED,NaN,3
2,400337,3650,17280,0,1200,01/01/2017,2050-12-31,01/01/1753,Male,DI,...,Executive C3,Full Time,Currently IN,UNSPECIFIED,Unspecified,Currently IN,Not Transferred,UNSPECFIED,NaN,3
3,400338,0,12000,0,14880,05/04/2010,2050-12-31,01/01/1753,Mr,DI,...,Supervisor,Full Time,Currently IN,ITI,Unspecified,Currently IN,Not Transferred,UNSPECFIED,NaN,3
4,400339,0,11990,"1096,00",12900,15/03/2022,2050-12-31,15/03/2019,Miss,DD,...,Employee E1,Full Time,Currently IN,PRO,Transfer,TRA,Voluntary,TWW,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,400044,0,11111,0,12330,29/03/2017,31/12/2050,01/01/1753,Mr,DI,...,Employee E3,Full Time,Currently IN,IIS,Unspecified,Currently IN,Not Transferred,UNSPECFIED,NaN,3
934,400046,0,10000,1338,10000,14/07/2021,31/12/2050,14/11/2017,Mrs,DI,...,Employee E2,Full Time,Currently IN,SBS,End_Contract,End of Contract,Organisation,IMI,NaN,3
935,400048,0,10990,1096,11900,15/03/2022,31/12/2050,15/03/2019,Miss,DI,...,Employee E1,Full Time,Currently IN,ATX,Transfer,AUTRE,Voluntary,IIS,NaN,3
936,400050,0,13770,366,11200,02/02/2022,31/12/2050,01/02/2021,Mr,DD,...,Executive C2,Full Time,Currently IN,AFA,Transfer,End of Contract,Organisation,EXM,NaN,3


#### Updates are Autosaved for this case
---------------------------------------

-----------------------------------------
## Note : UPDATE saving & Load versions + Move to Archive folders

- Move L_M to ArchiveBis Folder to avoid overwriting it. (cntr+F :    ArchiveBis\L_M    )
- Increment version in L_upd, df_upd ...

--- Done  ---